In [8]:
import pandas as pd
import altair as alt

## Analisis de Previo - contexto

In [3]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

### Ceses y Rotacion
* Los *ceses* son la cantidad de colaboradores que dejan de trabajar en la empresa en un determinado mes
* La *rotacion* es el % entre los colaboradores cesados y el total de colaboradores
* Saber cuales son los ceses y su comportamiento ayuda a tener una vision general, pero esto no es comparable entre areas, zonas o cargos.
    * Por qué? Pues un cargo puede tener 10 ceses pero como tiene en total 100 colaboradores su rotacion solo le impacta un 10%; sin embargo si un cargo tiene 5 ceses pero su total de colaboradores es de 10, su rotacion sera 50%  es decir mas impactante
* Debido a esto es importante analizar el comportamiento tomando en cuenta los Ceses y la rotacion que generan

In [4]:
ceses = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
ceses = pd.read_excel(
    ceses,
    usecols='E,I,O,P,S,T,W,X',
    dtype = {'CODIGO':str})

* Debido a un hecho extraordinario como una mudanza de una provincia a la capital los ceses del 2017 son atipicos, por lo se consideraran a partir del 2018
* Ademas el resto de bases parten desde este anio
* Es necesario transformar a inicio de mes la fecha de cese e ingreso porque es el intervalo que usare para comparar las temporalidades de otras bases

In [5]:
ceses.describe()

C:\Users\Anna\AppData\Local\Temp/ipykernel_13304/2583188236.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  ceses.describe()


,CODIGO,CARGO,INGRESO,CESE,Total o Temprana,Producto,TIPO,MOTIVO PRIMARIO
count,3467,3467,3467,3467,3467,3467,3467,3467
unique,3447,12,537,988,2,2,2,28
top,A86AAAAAA3,VENTA GRUPAL,2018-06-04 00:00:00,2020-01-02 00:00:00,-,GRUPAL,VOLUNTARIA,Mejor Oferta De Trabajo
freq,2,1984,61,38,2837,2246,2338,583
first,NaN,NaN,1999-10-11 00:00:00,2017-01-02 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2021-11-19 00:00:00,2021-12-31 00:00:00,NaN,NaN,NaN,NaN


In [6]:
# Tranformando y filtrando lo necesario para el analisis
ceses = ceses.query('CESE >= "2018-01-01"')
ceses['CESE'] = ceses['CESE'].to_numpy().astype('datetime64[M]')

* Analizo los ceses separando el producto por ser una variable que genera mucha diferencia en el negocio
* La rotacion en grupal en mas alta que en individual en numero de colaboradores cesados

In [10]:
ceses_prod = ceses.groupby(
    by=['Producto','CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count'))
# plot por producto
barra = alt.Chart(
    ceses_prod
).mark_bar(
    color = '#049DD9'
).encode(
    x = alt.X(
        'year(CESE):N',
        title = None),
    y = alt.Y(
        'sum(cesados):Q',
        title = None,
        axis = None)
).properties(
    width = 300,
    height = 200
)
texto = barra.mark_text(
    align = 'center',
    baseline ='top',
    dy = 5,
    color = '#04328C',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('sum(cesados)')
)
# plot barra  y texto
(barra+texto).facet('Producto').properties(
    title = 'Total de Ceses por Producto Acumulado por Año'
).configure_view(
    strokeWidth = 0 #quita el borde
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#04328C'
)


alt.FacetChart(...)

* Analizo el comportamineto de los ceses segun los cargos en cada tipo de producto

In [105]:
ceses_cargo = ceses.groupby(
    by=['Producto', 'CARGO', 'CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count')
    )

# alt.Chart(ceses_cargo).mark_bar().encode(
# x = 'year(CESE):N',
# y = 'sum(cesados):Q',
# color = alt.Color(
#     'CARGO:N',
#     scale= alt.Scale(scheme='category20')),
# column = 'Producto'
# )

# plot por producto
alt.Chart(
    ceses_cargo
).mark_bar(
    #color = '#049DD9'
).encode(
    y = alt.X(
        'year(CESE):N',
        title = None),
    x = alt.Y(
        'sum(cesados):Q',
        title = None,
        axis = None),
    color = alt.Color(
        'CARGO:O'),
        #scale= alt.Scale(scheme='category20')),
    column = 'Producto'
).properties(
    width = 300,
    height = 200,
    title = 'ño'
).configure_view(
    strokeWidth = 0 #quita el borde
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#04328C'
)
# texto = barra.mark_text(
#     align = 'center',
#     #baseline ='top',
#     #dy = 5,
#     #color = '#04328C',
#     #fontSize = 13,
#     fontWeight = 'bold'
# ).encode(
#     text = alt.Text('sum(cesados)')
# )
# plot barra  y texto
# (barra+texto).facet('Producto').properties(
#     title = 'ño'
# ).configure_view(
#     strokeWidth = 0 #quita el borde
# ).configure_axis(
#     labelFontSize = 15,
#     labelAngle = 0
# ).configure_title(
#     fontSize = 18,
#     color = '#04328C'
# )


alt.Chart(...)

In [ ]:
planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')

In [35]:
planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
entrevistas = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
bonos_venta_g = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
desempeno = extrae_drive()
reconocimientos = extrae_drive()
hijxs = extrae_drive()
datos_personales = extrae_drive()
recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
antg_oficina = extrae_drive()
gptw_oficina = extrae_drive()